In [380]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,StandardScaler
import src.utils.load_file as ld
from tensorflow.keras.activations import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [279]:
ohe = OneHotEncoder(sparse=False)
mms = MinMaxScaler()

### MFCC

In [304]:
X_train, X_test, y_train, y_test = ld.get_data(os.getcwd())

In [284]:
mms.fit(X_train)

X_train = mms.transform(X_train)
X_test  = mms.transform(X_test)

In [305]:
X_train = X_train.reshape(2976,431,-1)
X_test  = X_test.reshape(176,431,-1)

y1 = np.asarray(y_train,dtype='int').reshape(-1,1)
y_train = ohe.fit_transform(y1)

y2 = np.asarray(y_test,dtype='int').reshape(-1,1)
y_test = ohe.fit_transform(y2)

### MEL-SCALE


In [332]:
X_train, X_test, y_train, y_test = ld.get_data(os.getcwd(),i=10)

In [333]:
X_test.shape

(200, 5603)

In [311]:
os.getcwd()

'/Users/andrea/Documents/GitHub/audio-event-tagging/data'

In [327]:

t1 = np.load(os.getcwd()+'/audio/0_Animal.npy',allow_pickle=True).tolist()
t2 = np.load(os.getcwd()+'/audio/1_Animal.npy',allow_pickle=True).tolist()

In [338]:
t2['mel_spec'].shape

(128, 431)

In [ ]:
import zipfile

ohe = OneHotEncoder(sparse=False)


label_dict = {'Animal':0,
              'Humans':1,
              'Natural':2,
             }


zf = zipfile.ZipFile('features_.zip', 'r')

X= [np.load(zf.open(nam),allow_pickle =True).tolist()['mel_spec'] for nam in zf.namelist()]

y=np.asarray([label_dict[nam.split('_')[2].split('.')[0]] for nam in zf.namelist()],dtype='int').reshape(-1,1)

y = ohe.fit_transform(y)

In [381]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)

In [383]:
X_train

[array([[2.3913023e-01, 4.0367270e-01, 4.0663698e-01, ..., 4.0391350e-01,
         4.0338209e-01, 3.9623541e-01],
        [4.8475102e-01, 8.1830204e-01, 8.2431108e-01, ..., 8.1879014e-01,
         8.1771290e-01, 8.0322558e-01],
        [2.7298958e-03, 3.8027562e-02, 3.3750810e-02, ..., 3.1092495e-02,
         3.7277978e-02, 3.1830084e-02],
        ...,
        [6.6567281e-06, 3.1171890e-10, 3.1933614e-10, ..., 3.2967354e-10,
         9.9587250e-10, 1.2345583e-08],
        [9.9414128e-06, 1.6525803e-10, 1.2621952e-10, ..., 2.3995930e-10,
         1.4480850e-10, 9.6998001e-09],
        [1.1352785e-05, 1.5698713e-11, 2.0665391e-11, ..., 1.9423071e-11,
         8.9696592e-12, 2.6364095e-09]], dtype=float32),
 array([[3.8705772e-04, 1.6748410e-03, 2.1682449e-03, ..., 3.3591904e-02,
         4.4393316e-02, 1.2354540e-04],
        [7.8462105e-04, 3.3951411e-03, 4.3953410e-03, ..., 6.8095572e-02,
         8.9991570e-02, 2.5044411e-04],
        [2.8210398e-04, 2.6960523e-04, 6.4425415e-04, ...,

In [348]:
def get_model(select= 'mfcc'):
    
    choices = dict(mfcc=13, mel_spec = 128)
    
    assert select in choices.keys()
    
    input_size = choices[select]
    
    adam = Adam(learning_rate=0.01)
    nadam = Nadam(learning_rate=0.01)
    rms = RMSprop(learning_rate=0.01)
    
    inputs = Input(shape = (None, input_size))
    
    lstm = LSTM(32,recurrent_dropout=.4)
    encoder = Bidirectional(lstm)(inputs)
    encoder = Dropout(0.5)(encoder)
    
    
    decoder = Dense(10,activation='relu')(encoder)
    decoder = Dropout(0.50)(decoder)
    decoder = Dense(10,activation='relu')(decoder)
    decoder = Dropout(0.50)(decoder)
    
    output = Dense(3,activation='softmax')(decoder)

    model = Model(inputs = inputs,outputs = output)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer = rms)
    
    return model
    

get_model().summary()

Model: "model_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        [(None, None, 13)]        0         
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 64)                11776     
_________________________________________________________________
dropout_40 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_116 (Dense)            (None, 10)                650       
_________________________________________________________________
dropout_41 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_117 (Dense)            (None, 10)                110       
_________________________________________________________________
dropout_42 (Dropout)         (None, 10)                0  

In [301]:
model.fit(X_train,y_train,epochs=100,batch_size = 64, validation_data=(X_test,y_test))

Train on 2976 samples, validate on 176 samples
Epoch 1/100
2976/2976 [==============================] - 54s 18ms/sample - loss: 0.0147 - accuracy: 0.9970 - val_loss: 2.7424 - val_accuracy: 0.4659
Epoch 2/100
2976/2976 [==============================] - 55s 18ms/sample - loss: 0.0147 - accuracy: 0.9960 - val_loss: 2.7767 - val_accuracy: 0.4659
Epoch 3/100
2976/2976 [==============================] - 53s 18ms/sample - loss: 0.0137 - accuracy: 0.9963 - val_loss: 2.7627 - val_accuracy: 0.4602
Epoch 4/100
2976/2976 [==============================] - 53s 18ms/sample - loss: 0.0137 - accuracy: 0.9963 - val_loss: 2.8547 - val_accuracy: 0.4659
Epoch 5/100
2976/2976 [==============================] - 53s 18ms/sample - loss: 0.0154 - accuracy: 0.9960 - val_loss: 2.8184 - val_accuracy: 0.4432
Epoch 6/100
2976/2976 [==============================] - 55s 19ms/sample - loss: 0.0354 - accuracy: 0.9903 - val_loss: 2.7172 - val_accuracy: 0.4830
Epoch 7/100
2976/2976 [==============================] - 53

KeyboardInterrupt: 